In [33]:
#display options
from IPython.core.display import display, HTML, Math, Latex
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
display(HTML("<style>.container { width:99% !important; }</style>"))

 
import sympy as sp
from sympy import *
import functools 

e = sp.symbols('e0:1000', commutative=False)

basis = list(e)

def write(x):
    expand( standardize( expand( x )))

def getweight(x):
    return float((basis.index(x))/2 - 1)

def getindex(x):
    return int(2*(x + 1))

def getparity(x):
    L = unpack(x)
    parity=0
    for term in L:
        parity += int(basis.index(term))
    return int(parity%2)

def unpack(x):#check for multiplicity and write powers as distinct elements in list
    if type(x).__name__ == 'Mul': #check if passed argument is product e.g. e[1]*e[2]
        L = list(x.args)
        for term in L:
            if type(term).__name__ == 'Pow':#check if passed argument is product that contains powers e.g. e[1]*e[2]**2
                J = list(term.args) #returns [vector, power]
                power = J[1]
                base = J[0]
                firstL = L[:L.index(term)] #splitting the list to insert multiplicity many elements
                secondL = L[L.index(term) + 1: ]
                for i in range(power):
                    firstL.append(base)
                L = firstL + secondL
        return L
            
    if type(x).__name__ == 'Pow':#check if passed argument is just a power itself e.g. e[1]**2
        L = list(x.args) #returns [vector, power]
        base = L[0]
        power = L[1]
        L=[]
        for i in range(power):
            L += unpack(base)
        return L
            
    if type(x).__name__ == 'Symbol':#check if passed argument is single bases vector e.g. e[1]
        return [x]
    

def bracket(x,y):
    if x.is_number or y.is_number:
        return 0
    
    #bilinearity 
    if type(x).__name__ == 'Add':#split across sums on x
        L = list(x.args)
        J = []
        for term in L:
            J.append( bracket(term, y) )
        return sum(J)
        
    if type(y).__name__ == 'Add':#split across sums on y
        L = list(y.args)
        J = []
        for term in L:
            J.append( bracket(x, term) )
        return sum(J)
    
    if type(x).__name__ == 'Mul':    #pull out x coefficients
        L = list(x.args)
        if L[0].is_number:
            coeff = L[0]
            del L[0]
            return coeff*bracket( prod(L) , y )

    if type(y).__name__ == 'Mul':    #pull out y coefficients
        L = list(y.args)
        if L[0].is_number:
            coeff = L[0]
            del L[0]
            return coeff*bracket( x , prod(L) )
    
    
    #ad homomorphism property
    unpackedx = unpack(x)
    if len(unpackedx) > 1:#reducing product
        xreductionprod = prod(unpackedx[:-1])
        return bracket(xreductionprod, bracket(unpackedx[-1], y))
    
    #super Leibniz
    unpackedy = unpack(y)
    if len(unpackedy) > 1:
        yreductionprod = prod(unpackedy[1:])
        leibniz1 = bracket(x, unpackedy[0]) * yreductionprod
        leibniz2 = unpackedy[0] * bracket(x, yreductionprod)
        derivationParity = int((-1)**(getparity(x) * getparity(unpackedy[0])))
        return leibniz1 + derivationParity*leibniz2
        
    
    #base case
    weightx = getweight(x)
    weighty = getweight(y)
    combinedweight = weightx + weighty
    combinedindex = getindex(combinedweight)
    
    if weightx.is_integer() and weighty.is_integer():
        z = (weighty - weightx)*e[combinedindex]
    if weightx.is_integer() and not weighty.is_integer():
        z = (weighty - weightx/2)*e[combinedindex]
    if not weightx.is_integer() and weighty.is_integer():
        z = -1*(weightx - weighty/2)*e[combinedindex]
    if not weightx.is_integer() and not weighty.is_integer():
        z = 2*e[combinedindex]
    return z

def standardize(x):
    if type(x).__name__ == 'Symbol' or x.is_number:
        return x
    
    if type(x).__name__ == 'Add':#split across sums on x
        L = list(x.args)
        J = []
        for term in L:
            J.append( standardize(term) )
        return sum(J)

    if type(x).__name__ == 'Mul':    #pull out x coefficients
        L = list(x.args)
        if L[0].is_number:
            coeff = L[0]
            del L[0]
            return coeff*standardize( prod(L) )
    
    L = unpack(x)
    ordercheck = 0
    indexforswap = 0
    
    for i in range(len(L)-1):#store index of point for swap, exit loop and then call swap with bracket   
        if basis.index(L[i]) > basis.index(L[i+1]):
            indexforswap = i
            ordercheck+=1
            break
        
        if basis.index(L[i]) == basis.index(L[i+1]) and basis.index(L[i])%2 == 1:
            L[i] = .5*bracket(L[i], L[i+1])
            del L[i+1]
            return standardize( prod(L) )
            
    if ordercheck == 0: 
        return x
    
    if ordercheck != 0:
        firstJ= []
        secondJ = []
        for j in L[:i]:
            firstJ.append(j)
        firstJ.append(bracket(L[i], L[i+1])) #insert bracket into new list
            
        if indexforswap + 2 < len(L):
            for j in L[indexforswap+2:]:
                secondJ.append(j) #get second half of list
                    
        J = firstJ + secondJ
        parity = int( (-1)**( getparity(L[indexforswap] ) * getparity( L[indexforswap+1]) ) )
        L[indexforswap], L[indexforswap+1] = L[indexforswap+1], L[indexforswap] #finally swap

        return parity*standardize( prod(L) ) + standardize( prod(J) ) #run through again to reorder



#######################################################################################################################################################################
#Defining some commonly used objects.

#Step operators
S3half = (2*e[2] - 1)*e[6] - 1.5*(e[3]*e[5] + e[4]*e[4]) #weight 3/2 step operator
S2 = (4*e[2] - 2)*e[2] - 3*e[4]**2 - 3*e[3]*e[5] #weight 2 step operator
S5half = 0 #weight 5/2 step operator

#Quadratic Terms
Qs = e[2]**2 + .5*e[2] + .5*e[1]*e[3] - e[0]*e[4] #osp Casimir
Q = e[2]**2 - e[2] - e[4]*e[0] #sl2 Casimir
Lambda = e[2] - e[3]*e[1]
T = Lambda - 1/4 #ghost
LWVn1half = bracket(e[1], T) #LWV of weight -1/2
Te2 = bracket(e[6], T) #T^{e_2}
Qse2 = bracket(e[6], Qs)#ad(e_2)(Casimir of osp)

#Climbing the F_0 in S^2 starting with Z_{1/2}, which drops to Casimir under e_{-1/2}
Z1half = e[4]*e[1] - 2*e[2]*e[3] + e[0]*e[5] 
Z1 = bracket(e[3], Z1half)
Z3half = bracket(e[3], Z1)
Z2 = bracket(e[3], Z3half)

#Cubic Terms
Y0 = expand(4*Qs*(e[2] - 1/4) + .5*Z1half*e[1] + Z1*e[0])#LWV of weight 0 in S^3
Yn1half = expand(2*Qs*e[1] + Z1half*e[0]) #LWV of weight -1/2 in S^3
#######################################################################################################################################################################

In [3]:
#Example calculation showing that Z_{1/2} drops to a multiple of Casimir of osp under ad(e_{-1/2}) in S^2
b = bracket(e[1], Z1half)
sb = simplify(standardize(b))
standqs = standardize(Qs)
sbdiff = simplify(sb + 4*standqs)
sbdiff

0

In [4]:
#Example calculation showing that Te2 = T^{e_2} straddles the F_{3/2} and F_2 in S^2.
simplify(standardize(bracket(e[1], bracket(e[1], Te2))))

#Example calculation verifying that the weight 1 vector dropping into the F_-1/2 of S^2 is T^{e_3/2}
t1 = expand( standardize( bracket(e[5], LWVn1half)))
p0 = expand( bracket(e[3], LWVn1half) )
p1half = standardize( expand( bracket(e[3], p0) ) )
p1 = standardize( expand( bracket(e[3], p1half) ) )#gives 0

expand( standardize( expand( bracket(e[1], t1) ) ) ) 
expand( p1half )

0

4.0*e1*e4 - 4.0*e2*e3 - 1.0*e3

2.0*e1*e4 - 2.0*e2*e3 - 0.5*e3

In [5]:
#Constructing vector in S^3 that straddles an F_1 and a F_1/2
expr = expand(Te2*e[0])
drop1 = bracket(e[1], expr)
drop2 = bracket(e[1], drop1)
simplify(standardize(drop2))

0

In [6]:
#Deducing LWV of weight 0 in S^3, Y0, via computing one piece at a time and then finding a solution.
p1 = expand( Qs*e[2] )
p2 = expand( Z1half*e[1] )
p3 = expand( Z1*e[0] )
br1 = expand( bracket( e[1], p1 ) )
br2 = expand( bracket( e[1], p2 ) )
br3 = expand( bracket( e[1], p3 ) )
br1, br2, br3

(-0.5*e0*e4*e1 + 0.25*e1*e2 - 0.5*e1*e2**2 + 0.25*e1*e3*e1 + 0.25*e2*e1 + 0.5*e2*e1*e2 + 0.5*e2**2*e1,
 2*e0*e4*e1 - 2*e0*e5*e0 - 1.0*e1*e3*e1 + 4*e2*e3*e0 - 4*e2**2*e1 + 1.0*e3*e1**2 + 2*e4*e0*e1 - 2*e4*e1*e0,
 1.0*e0*e5*e0 - 2.0*e2*e3*e0 + 1.0*e4*e1*e0)

In [7]:
#From previous cell, conclude that coeff1 = 4, coeff2 = .5 and coeff3 = 1 is a solution based only off e_0*e_1*e_4 term at level of symbol. Check below. Y0 is defined above in the first box.
br = bracket(e[1], Y0)
expbr = expand(br)
stbr = standardize(expbr)
expand(stbr)

0

In [8]:
#Getting weight 1 vector that drops to Y0.
br = bracket(e[3], bracket(e[3], Y0) )
expr = standardize( expand(br) )
Y1 = simplify(expr)
Y1

-1.5*e0*e1*e7 - 8.0*e0*e2*e6 + 1.5*e0*e3*e5 + 6.0*e0*e4**2 + 4.0*e0*e6 + 2.0*e0**2*e8 + 3.0*e1*e2*e5 - 3.0*e1*e3*e4 - 1.5*e1*e5

In [9]:
#Next, compute LWV of weight -1/2 in S^3 using same tactic. Then check.
p1 = expand( Qs*e[1] )
p2 = expand( Z1half*e[0] )
br1 = expand( bracket( e[1], p1 ) )
br2 = expand( bracket( e[1], p2 ) )
temp = expand( 2*expand( standardize(br1) ) + expand( standardize(br2) ) ) #gives zero

check = standardize( expand (bracket(e[1], Yn1half)) )
expand(check)

0

In [10]:
#Getting weight 1 vector in the F_-1/2 of S^3
W1 =  expand( standardize( expand( bracket(e[3]**3, Yn1half))))
V1 = expand( standardize( bracket(e[5], LWVn1half)))

expand( W1 )#weight 1 vector in F_-1/2 of S^3
expand( Y1 )#weight 1 vector in F_0 of S^3
expand( V1 )#weight 1 vector in F_-1/2 of S^2
expand( -2*(W1-Y1) )

-2.0*e0*e1*e7 - 8.0*e0*e2*e6 + 2.0*e0*e3*e5 + 6.0*e0*e4**2 + 4.0*e0*e6 + 2.0*e0**2*e8 + 4.0*e1*e2*e5 - 4.0*e1*e3*e4 - 2.0*e1*e5

-1.5*e0*e1*e7 - 8.0*e0*e2*e6 + 1.5*e0*e3*e5 + 6.0*e0*e4**2 + 4.0*e0*e6 + 2.0*e0**2*e8 + 3.0*e1*e2*e5 - 3.0*e1*e3*e4 - 1.5*e1*e5

4*e0*e6 - 3.0*e1*e5 - 4*e2*e4 - 1.0*e4

1.0*e0*e1*e7 - 1.0*e0*e3*e5 - 2.0*e1*e2*e5 + 2.0*e1*e3*e4 + 1.0*e1*e5

In [98]:
B = expand( standardize( expand( bracket(e[3], Yn1half))))
T3 = expand(standardize( expand( 2*(B - Y0) )))
#hypothesis: no powers of T can give cubic, so this is indeed a "cubic ghost" and not simply an expression in T, MAYBE. T3 is NOT in the anti-center or center.
#expand(standardize(expand(T3**2)))#T3 square is not degree 6, it is degree 4
V = expand(-1*(T3+expand(standardize(expand(Q)))))
Vsquared = expand(standardize(expand(V**2)))#V squares to degree 4, despite being cubic
W = expand(-1*(T3+2*expand(standardize(expand(Qs)))))
V
W
Qs
T3

1.0*e0*e1*e5 - 2.0*e1*e2*e3 + 1.0*e2**2

1.0*e0*e1*e5 + 1.0*e0*e4 - 2.0*e1*e2*e3 - 1.0*e1*e3

-e0*e4 + 0.5*e1*e3 + 0.5*e2 + e2**2

-1.0*e0*e1*e5 + 1.0*e0*e4 + 2.0*e1*e2*e3 - 1.0*e2 - 2.0*e2**2